# Imports

In [10]:
import os
import tkinter as tk
from tkinter import filedialog
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from tkinter.filedialog import askdirectory
from scipy.signal import savgol_filter, find_peaks
from scipy.integrate import trapezoid
from scipy.signal import butter, filtfilt, find_peaks
from numpy.fft import fft
from scipy.stats import ttest_1samp, shapiro
import statsmodels.stats.multitest as multitest
from scipy.stats import wilcoxon

# Main

## Load data

In [11]:
# Load the aggregated file on which to do the statistics

file_path = filedialog.askopenfilename()
aggregated_data=pd.read_excel(file_path)


## Data reorganization

In [12]:
Aggregated_Columns=aggregated_data.columns

#Find number and list of different exp_wells
Exp_column=aggregated_data['Name'].values.flatten()
ExpID=[]
Exp_wellID=[]
UniqueExpWell=[]

for i in range(0,len(Exp_column)):
    ExpID.append(Exp_column[i].split('_')[0]) #we get the experiment
    Exp_well=ExpID[i] + '_'+ aggregated_data.loc[i,'Well'] #we get the tissue
    Exp_wellID.append(Exp_well) #Parameter Exp_Tissue to follow accross the different concentrations

# Add new columns to dataframe
aggregated_data['Exp']=ExpID
aggregated_data['ExpWell']=Exp_wellID

print(np.unique(Exp_wellID))
UniqueExpWell=np.unique(Exp_wellID)


#Find number and list of different concentrations
Conc = sorted(aggregated_data['Concentration'].unique())  # Sort concentrations

# Create Excel writer for output
if file_path.endswith('.xlsx'):
    output_path = file_path.replace('.xlsx', '_reorganized.xlsx')
elif file_path.endswith('.xls'):
    output_path = file_path.replace('.xls', '_reorganized.xlsx')
else:
    output_path = file_path + '_reorganized.xlsx'

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:

    #Rearrange data
    for col in Aggregated_Columns:

        # Skip non-parameter columns
        if col in ['Name', 'Exp', 'Well', 'ExpWell', 'Concentration']:
            continue

        print(f"\nProcessing parameter: {col}")

        # Create sheet column names
        Sheet_columns=['Name','Exp','Well']
        col_name=col.replace(' ','_')
        
        for c in Conc:
            Sheet_columns.append(f'Conc_{c}')

        # Create empty dataframe for this parameter
        Parameter_sheet=pd.DataFrame(columns=Sheet_columns)
        
        # Process each unique experiment-well combination
        rows_data=[]
        for exp_well in UniqueExpWell:
            # Get the metadata for this exp_well (using first occurrence)
            exp_well_data = aggregated_data[aggregated_data['ExpWell'] == exp_well]
            
            if len(exp_well_data) <2:
                continue
                    
            # Get basic info (should be same for all concentrations of this exp_well)
            name = exp_well_data['Name'].iloc[1]
            exp = exp_well_data['Exp'].iloc[0]
            well = exp_well_data['Well'].iloc[0]

            # Create row data starting with metadata
            row_data = {
                'Name': name,
                'Exp': exp,
                'Well': well
            }

            # Get parameter values for each concentration
            for c in Conc:
                conc_data = exp_well_data[exp_well_data['Concentration'] == c]
                if len(conc_data) > 0:
                            param_value = conc_data[col].iloc[0]  # Take first value if multiple
                else:
                    param_value = np.nan  # No data for this concentration
                
                row_data[f'Conc_{c}'] = param_value

            rows_data.append(row_data)
        
        # Create dataframe from collected data
        Parameter_sheet = pd.DataFrame(rows_data)
        
        # print(f"Parameter sheet for {col}:")
        # print(Parameter_sheet.head())

        # Clean column name for Excel sheet name (remove special characters, limit length)
        sheet_name = col_name.replace('/', '_').replace('\\', '_').replace('*', '_')
        sheet_name = sheet_name.replace('[', '_').replace(']', '_').replace(':', '_')
        sheet_name = sheet_name.replace('?', '_').replace('|', '_')[:31]  # Excel limit
        
        # Write to Excel sheet
        Parameter_sheet.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Sheet '{sheet_name}' created with {len(Parameter_sheet)} rows")

print(f"\nReorganized data saved to: {output_path}")

# Get statistics

# Create Excel writer for output
if file_path.endswith('.xlsx'):
    StatPath = file_path.replace('.xlsx', '_Stats.xlsx')
elif file_path.endswith('.xls'):
    StatPath = file_path.replace('.xls', '_Stats.xlsx')
else:
    StatPath = file_path + '_Stats.xlsx'

with pd.ExcelWriter(StatPath, engine='openpyxl') as writer:
    for col in Aggregated_Columns:
        if col in ['Name', 'Exp', 'Well', 'ExpWell', 'Concentration']:
            continue
        
        # Read the sheet we just created
        sheet_name = col.replace(' ', '_').replace('/', '_')[:31]
        param_data = pd.read_excel(output_path, sheet_name=sheet_name)
        stats_columns=['Conc_1', 'Conc_2', 'Conc_3', 'Conc_4']
        print(param_data.columns.tolist())
        stats_data=param_data[stats_columns]

        # Perform Shapiro-Wilk test for normality
        print(f"Shapiro-Wilk Normality Test Results for {sheet_name}:")
        NormalGlobal=True
        for conc, values in stats_data.items():
            stat, p_value = shapiro(values,nan_policy='omit')
            if p_value < 0.05:
                NormalGlobal=False
            print(f"{conc}: W = {stat:.4f}, p-value = {p_value:.4f}, Normal = {p_value > 0.05}")
        
        print(f"NormalGolbal={NormalGlobal}")

        results = pd.DataFrame(columns=['Concentration','Test','Stat','p-value','Significance'])
        tests=[]
        statval=[]
        pval=[]
        significance=[]
        concentration=[]

        # # If normal, t-test
        # if NormalGlobal==True:
        #     # Perform one-sample t-tests against baseline (1)
        #     alpha = 0.05
        #     n_tests = len(stats_data) #length of a column without NaNs
        #     bonferroni_alpha = alpha / n_tests
            

        #     print("\nOne-Sample T-Test Results (vs. baseline = 1):")
        #     for conc, values in stats_data.items():
        #         stat, p_value = ttest_1samp(values, popmean=1, nan_policy='omit')
        #         concentration.append(conc)
        #         tests.append('T-test')
        #         statval.append(stat)
        #         pval.append(p_value)
        #         significance.append(p_value < bonferroni_alpha)
        #         print(f"{conc}: t = {stat}, p-value = {p_value}, Significant = {p_value < bonferroni_alpha}")

        #     # Bonferroni correction
        #     _, pvals_corrected, _, _ = multitest.multipletests(pval, alpha=alpha, method='bonferroni')
        #     print("\nBonferroni-Corrected p-values:")
        #     for conc, p_corr in zip(stats_data.keys(), pvals_corrected):
        #         print(f"{conc}: Corrected p-value = {p_corr}, Significant = {p_corr < alpha}")


        # If not normal, wilcoxon
        # if NormalGlobal==False:
            #Perform wilcoxon test
        for conc, values in stats_data.items():
            stat, p_value = wilcoxon(values, param_data['Conc_-1'], nan_policy='omit')
            concentration.append(conc)            
            tests.append('Wilcoxon')
            statval.append(stat)
            pval.append(p_value)
            significance.append(p_value < 0.05)
            print(f"{conc} Wilcoxon: statistic = {stat}, p-value = {p_value}, Significant = {p_value < 0.05}")

        results['Concentration']=concentration
        results['Test']=tests
        results['Stat']=statval
        results['p-value']=pval
        results['Significance']=significance

        

        #   Clean column name for Excel sheet name (remove special characters, limit length)
        sheet_name2 = sheet_name.replace(' ', '_').replace('\\', '_').replace('*', '_')
        sheet_name2 = sheet_name2.replace('[', '_').replace(']', '_').replace(':', '_')
        sheet_name2 = sheet_name2.replace('?', '_').replace('|', '_')[:31]  # Excel limit

    
        results.to_excel(writer, sheet_name=sheet_name2, index=False)




# # Create CSV files for Prism import
# csv_folder = file_path.rsplit('.', 1)[0] + '_CSV_for_Prism'
# import os
# os.makedirs(csv_folder, exist_ok=True)

# for col in Aggregated_Columns:
#     if col in ['Name', 'Exp', 'Well', 'ExpWell', 'Concentration']:
#         continue
    
#     # Read the sheet we just created
#     sheet_name = col.replace(' ', '_').replace('/', '_')[:31]
#     param_data = pd.read_excel(output_path, sheet_name=sheet_name)
    
#     # Create Prism format
#     prism_data = param_data.drop(['Name', 'Exp', 'Well'], axis=1, errors='ignore')
#     row_labels = param_data['Exp'].astype(str) + '_' + param_data['Well'].astype(str)
#     prism_data.insert(0, 'Sample', row_labels)
    
#     # Save as CSV
#     csv_path = os.path.join(csv_folder, f"{sheet_name}.csv")
#     prism_data.to_csv(csv_path, index=False)
#     print(f"CSV created: {csv_path}")

['260824_E3b' '260824_E3d' '260824_E3e' '260824_E3f' '260824_E3h'
 '260824_F3b' '260824_F3d' '260824_F3e' '260824_F3g' '260824_F3h'
 '260824_G3a' '260824_G3c' '260824_G3d' '260824_G3f' '260824_G3g'
 '260824_G3h' '260824_G3i' '310724_E3d' '310724_E3e' '310724_E3h'
 '310724_F3d' '310724_F3e' '310724_F3h' '310724_F3i' '310724_G3c'
 '310724_G3d' '310724_G3e' '310724_G3g' '310724_G3h' '310724_G3i']

Processing parameter: Total video time
Sheet 'Total_video_time' created with 30 rows

Processing parameter: SNR
Sheet 'SNR' created with 30 rows

Processing parameter: Initial Youngs Modulus
Sheet 'Initial_Youngs_Modulus' created with 30 rows

Processing parameter: Days in culture
Sheet 'Days_in_culture' created with 30 rows

Processing parameter: Period (s)
Sheet 'Period_(s)' created with 30 rows

Processing parameter: Beating Frequency (Hz)
Sheet 'Beating_Frequency_(Hz)' created with 30 rows

Processing parameter: Interpeak irregularity (s)
Sheet 'Interpeak_irregularity_(s)' created with 30 ro

c:\Users\inesb\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\inesb\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\inesb\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\inesb\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
c:\Users\inesb\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.sta

['Name', 'Exp', 'Well', 'Conc_-1', 'Conc_1', 'Conc_2', 'Conc_3', 'Conc_4']
Shapiro-Wilk Normality Test Results for Interpeak_irregularity_(s):
Conc_1: W = 0.9603, p-value = 0.3747, Normal = True
Conc_2: W = 0.4426, p-value = 0.0000, Normal = False
Conc_3: W = 0.7639, p-value = 0.0001, Normal = False
Conc_4: W = 0.6852, p-value = 0.0010, Normal = False
NormalGolbal=False
Conc_1 Wilcoxon: statistic = 55.0, p-value = 0.0007440149784088135, Significant = True
Conc_2 Wilcoxon: statistic = 129.0, p-value = 0.5645889043807983, Significant = False
Conc_3 Wilcoxon: statistic = 59.0, p-value = 0.0041751861572265625, Significant = True
Conc_4 Wilcoxon: statistic = 0.0, p-value = 0.00390625, Significant = True
['Name', 'Exp', 'Well', 'Conc_-1', 'Conc_1', 'Conc_2', 'Conc_3', 'Conc_4']
Shapiro-Wilk Normality Test Results for N_twitch:
Conc_1: W = 0.5970, p-value = 0.0000, Normal = False
Conc_2: W = 0.4012, p-value = 0.0000, Normal = False
Conc_3: W = 0.6723, p-value = 0.0000, Normal = False
Conc_4: 